In [ ]:
#install
#@markdown To set up Simple Stable, click on the triangle play button on the left. The interface will appear below.
#@markdown
#@markdown To hide large or annoying code on the left, double-click on this text.
#@markdown
#@markdown To open the interface in a new tab or window, click on the link after "Running on public URL".\
#@markdown When loading a new model for the first time, it may take a few minutes to download, but switching between already downloaded models will be faster.\
#@markdown Please note that time estimates may not be accurate.

starting_model_to_load = "Stable Diffusion 1.5" #@param ["Stable Diffusion 1.4","Stable Diffusion 1.5","Stable Diffusion 2.0 (512)","Stable Diffusion 2.0 (768)","Stable Diffusion 2.1 (512)","Stable Diffusion 2.1 (768)","Dreamlike Diffusion","Openjourney","Openjourney V2 Beta","Oldjourney Lite","Oldjourney Ultra","seek.art_MEGA","Sygil Diffusion","Analog Diffusion","Dreamlike Photoreal","Anything v3.0","Anything v4.0","Waifu Diffusion 1.4 (Epoch 1 Test Version)","Plat Diffusion (Waifu 1.4 finetune)","AnimeChanStyle","Inkpunk Diffusion","Vivid Watercolors","Future Diffusion","Gigafractal Diffusion (768)","Van Gogh Diffusion","Corporate Memphis","Pokemon Diffusion","Pokemon Sugimori SD 2.1","Robo Diffusion","Classic Animation Diffusion","Modern Animation Diffusion","Ghibli Diffusion","Furry Diffusion"] {type:"string"}

download_a_collection_of_text_inversion_embeddings_and_load_them = False #@param{type: "boolean"}

attention_slicing = False
test_mode = False

try:
  import torch
  import src
  %cd /content/simplest-stable/
except ImportError as e:
  import os, subprocess
  %cd /content/
  subprocess.run(["git", "clone", "https://github.com/cadaeix/simplest-stable.git"])
  %cd /content/simplest-stable/
  if test_mode:
    subprocess.run(["git", "checkout", "test-branch"])
  subprocess.run(["pip3", "install", "-r", "requirements.txt"])
  outputs_path = "/content/images/"
  subprocess.run(["mkdir", "-p", outputs_path])
  print(f"Outputs will be saved to {outputs_path}.")
  downloaded_embeds_path = None

import os, subprocess, sys, random, math, json, glob

# Pillow occasionally has odd import errors, the below fix them
import PIL.Image
if not hasattr(PIL.Image, 'Resampling'):  # Pillow<9.0
  PIL.Image.Resampling = PIL.Image

import PIL._util
if not hasattr(PIL._util, 'is_path'):
  from pathlib import Path
  def is_path(f):
    return isinstance(f, (bytes, str, Path))
  PIL._util.is_path = is_path

import PIL.TiffTags
if not hasattr(PIL.TiffTags, 'IFD'):
  PIL.TiffTags.IFD = 13
if not hasattr(PIL.TiffTags, 'LONG8'):
  PIL.TiffTags.IFD = 16

from datetime import datetime
from src.loading import prepare_pipe
from src.process import process_and_generate
from src.utils import free_ram
from src.gradio_ui import main
from IPython.display import display, clear_output
sys.path.append(".")

if download_a_collection_of_text_inversion_embeddings_and_load_them:
  downloaded_embeds_path = "/content/embeds/"
  if not os.path.exists(downloaded_embeds_path):
    subprocess.run(["mkdir", "-p", downloaded_embeds_path])
    with open('src/resources/embeddings_links.json') as embeddingfile:
      embed_dict = json.load(embeddingfile)
    for embed in embed_dict["embeddings"]:
      subprocess.run(["wget", "-O", os.path.join(downloaded_embeds_path, embed["filename"]), embed["download_url"]])

try:
  outputs_folder
except NameError as e:
  outputs_path = "images/"
  if not os.path.exists(outputs_path):
      os.mkdir(outputs_path)
      
  outputs_folder = os.path.join(outputs_path, datetime.now().strftime("%Y_%m_%d"))

try:
  custom_models_folder
  custom_models_path = custom_models_folder if os.path.exists(custom_models_folder) else None
except NameError as e:
  custom_models_path = None

try:
  custom_embeddings_folder
  embeddings_path = custom_embeddings_folder if os.path.exists(custom_embeddings_folder) else None
except NameError as e:
  embeddings_path = None

clear_output(wait=False)

main(starting_model_to_load, outputs_folder, custom_models_path, embeddings_path, downloaded_embeds_path)